In [45]:
import pandas as pd
import numpy as np

test = pd.read_csv('https://raw.githubusercontent.com/rogovich/Data/master/data/titanic/test.csv')
train = pd.read_csv('https://raw.githubusercontent.com/rogovich/Data/master/data/titanic/train.csv')
submission = pd.read_csv('https://raw.githubusercontent.com/rogovich/Data/master/data/titanic/gender_submission.csv')

In [46]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
test.head() # выборка для предсказаний

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [48]:
submission.head() # формат для овтета Kaggle

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# Baseline-модели
1. Предсказание самого частотного класса / Предсказание средней метрики

In [49]:
train.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

### Baseline

In [50]:
submission.Survived = 0

In [51]:
submission.to_csv('2022_baseline.csv', index=False)

### Random

In [52]:
submission.Survived = np.random.choice([0, 1], len(submission.Survived))

In [53]:
submission.to_csv('2022_random.csv', index=False)

### Gender

In [54]:
submission.Survived = test.Sex.apply(lambda x: 1 if x=='female' else 0)

### Siblings

In [55]:
submission.Survived = np.where((test.Sex == 'female') | ((test.Parch == 0) & (test.SibSp == 0)), 1, 0)

In [56]:
np.where((test.Sex == 'female') | ((test.Parch == 0) & (test.SibSp == 0)), 1, 0).sum()

337

### Параметры для дерева

1. max_depth - максимальная глубина дерева
2. max_features - ограничение на количество признаков для разбиения
3. min_samples_leaf - минимальное число объектов в листе (чтобы не уходить энергозатратные детали)

In [57]:
from sklearn.tree import DecisionTreeClassifier

In [58]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [83]:
print(test.isna().sum())
print(train.isna().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Title            0
dtype: int64
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Title            0
dtype: int64


In [74]:
def sep(name):
    return name.split(', ')[1].split('. ')[0]
test['Title'] = test.Name.apply(sep)
test.groupby('Title').Age.transform('mean')

0      32.000000
1      38.903226
2      32.000000
3      32.000000
4      38.903226
         ...    
413    32.000000
414    39.000000
415    32.000000
416    32.000000
417     7.406471
Name: Age, Length: 418, dtype: float64

In [75]:
def sep(name):
    return name.split(', ')[1].split('. ')[0]
train['Title'] = train.Name.apply(sep)
train.groupby('Title').Age.transform('mean')

0      32.368090
1      35.898148
2      21.773973
3      35.898148
4      32.368090
         ...    
886    43.166667
887    21.773973
888    21.773973
889    32.368090
890    32.368090
Name: Age, Length: 891, dtype: float64

In [76]:
test['Age'] = test.Age.fillna(test.groupby('Title').Age.transform('mean'))
test['Age'] = test.Age.fillna(test.Age.mean())

In [77]:
train['Age'] = train.Age.fillna(train.groupby('Title').Age.transform('mean'))
train['Age'] = train.Age.fillna(train.Age.mean())

In [79]:
X_train = train[['Age', 'SibSp', 'Parch']]
X_test = test[['Age', 'SibSp', 'Parch']]
y_train = train['Survived']

In [85]:
model = DecisionTreeClassifier(max_leaf_nodes=10)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=10)

In [81]:
submission['Survived'] = model.predict(X_test)
# сохраняем в csv и загружаем на Kaggle